![](https://camo.githubusercontent.com/7e60cef1bb1a2ac4b7b2457c9a13a6ef22ff59e1/68747470733a2f2f7773312e73696e61696d672e636e2f6c617267652f303036744e6337396c7931666d70766a37667864316a333076623065797a71662e6a7067)

In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [2]:
def conv_block(in_channel, out_channel):
    layer = nn.Sequential(
        nn.BatchNorm2d(in_channel),
        nn.ReLU(True),
        nn.Conv2d(in_channel, out_channel, 3, padding=1, bias=False)
    )
    return layer

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, in_channel, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        block = []
        channel = in_channel
        for i in range(num_layers):
            block.append(conv_block(channel, growth_rate))
            channel += growth_rate
        
        self.net = nn.Sequential(*block)
    
    def forward(self, x):
        for layer in self.net:
            out = layer(x)
            x = torch.cat((out, x), dim=1)
        return x

In [4]:
test_net = DenseBlock(3, 12, 3)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print("input shape: {} x {} x {}".format(test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print("output shape: {} x {} x {}".format(test_y.shape[1], test_y.shape[2], test_y.shape[3]))

input shape: 3 x 96 x 96
output shape: 39 x 96 x 96


In [5]:
def transition(in_channel, out_channel):
    trans_layer = nn.Sequential(
        nn.BatchNorm2d(in_channel),
        nn.ReLU(True),
        nn.Conv2d(in_channel, out_channel, 1),
        nn.AvgPool2d(2, 2)
    )
    return trans_layer

In [6]:
test_net = transition(3, 12)
test_x = Variable(torch.zeros(1, 3, 96, 96))
print("input shape: {} x {} x {}".format(test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print("output shape: {} x {} x {}".format(test_y.shape[1], test_y.shape[2], test_y.shape[3]))

input shape: 3 x 96 x 96
output shape: 12 x 48 x 48


In [7]:
# 定义 DenseNet
class DenseNet(nn.Module):
    def __init__(self, in_channel, num_classes, growth_rate=32, block_layers=[6, 12, 24, 16]):
        super(DenseNet, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channel, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(3, 2, padding=1)
        )

        channels = 64
        block = []
        for i, layers in enumerate(block_layers):
            block.append(DenseBlock(channels, growth_rate, layers))
            channels += layers * growth_rate
            if i != len(block_layers) - 1:
                block.append(transition(channels, channels // 2))
                channels = channels // 2
        self.block2 = nn.Sequential(*block)
        self.block2.add_module("bn", nn.BatchNorm2d(channels))
        self.block2.add_module("relu", nn.ReLU(True))
        self.block2.add_module("avg_pool", nn.AvgPool2d(3))

        self.classifier = nn.Linear(channels, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x


In [8]:
test_net = DenseNet(3, 10)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

output: torch.Size([1, 10])


In [9]:
def data_tf(x):
    x = x.resize((96, 96), 2)
    x = np.array(x, dtype="float32") / 255
    x = (x - 0.5) / 0.5
    x = x.transpose((2, 0, 1))  # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x

In [14]:
train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10("./data", train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = DenseNet(3, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

In [15]:
from utils import train

train(net, train_data, test_data, 20, optimizer, criterion)

D:\Workspace\Samples\pytorch_lab\chapter03_cnn\utils.py:54: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im.cuda(), volatile=True)
D:\Workspace\Samples\pytorch_lab\chapter03_cnn\utils.py:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label.cuda(), volatile=True)


Epoch 0. Train Loss: 1.368672, Train Acc: 0.509811, Valid Loss: 1.552130, Valid Acc: 0.474585, Time 00:02:23
Epoch 1. Train Loss: 0.913808, Train Acc: 0.677330, Valid Loss: 1.135782, Valid Acc: 0.619759, Time 00:02:27
Epoch 2. Train Loss: 0.702387, Train Acc: 0.758232, Valid Loss: 0.985468, Valid Acc: 0.667326, Time 00:02:27
Epoch 3. Train Loss: 0.572325, Train Acc: 0.801730, Valid Loss: 0.725179, Valid Acc: 0.749011, Time 00:02:27
Epoch 4. Train Loss: 0.477660, Train Acc: 0.835997, Valid Loss: 0.862192, Valid Acc: 0.727848, Time 00:02:27
Epoch 5. Train Loss: 0.397709, Train Acc: 0.864270, Valid Loss: 0.859988, Valid Acc: 0.716871, Time 00:02:27
Epoch 6. Train Loss: 0.340893, Train Acc: 0.883272, Valid Loss: 0.730948, Valid Acc: 0.769976, Time 00:02:27
Epoch 7. Train Loss: 0.277668, Train Acc: 0.906450, Valid Loss: 0.903186, Valid Acc: 0.738528, Time 00:02:27
Epoch 8. Train Loss: 0.228544, Train Acc: 0.922974, Valid Loss: 0.718776, Valid Acc: 0.782338, Time 00:02:27
Epoch 9. Train Loss